<a href="https://colab.research.google.com/github/Pujithakalluri/FMML-LAB-MODULES/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

# ANS TO QUESTION 1

The percentage of the validation set in your dataset can have a significant impact on the accuracy of the validation set and the overall training process. Here's how it is affected when you increase or reduce the size of the validation set:

Increasing the Percentage of Validation Set:

Advantages:

A larger validation set provides more data for evaluating the model's performance, which can lead to a more reliable estimate of how well the model generalizes to unseen data.
It can help in detecting overfitting more effectively. A larger validation set makes it easier to identify if the model is memorizing the training data (overfitting) because the model has less data overlap between training and validation.
Disadvantages:

You have fewer data points available for training, which can hinder the model's ability to learn from the data effectively. This can lead to underfitting if the training dataset is small.
Training on a smaller portion of the data can potentially limit the model's capacity to capture complex patterns in the data.
Reducing the Percentage of Validation Set:

Advantages:

You have more data available for training, which can be advantageous when dealing with limited data. A larger training set allows the model to potentially learn more complex patterns.
Training on a larger portion of the data can lead to better model performance if the dataset is small and the model has sufficient capacity to learn from it.
Disadvantages:

The validation accuracy may become less reliable as it is based on a smaller subset of the data. The validation set's accuracy may not be a good indicator of how well the model generalizes to unseen data.
Overfitting can be harder to detect if the model has a larger portion of the data for training, as there's more potential for data leakage between the training and validation sets.
The choice of the percentage of the validation set should be based on the size and nature of your dataset, as well as the goals of your machine learning task. Typically, a common split ratio is 70-80% for training and 20-30% for validation, but this can vary based on the specific problem. If you have a very large dataset, you can afford a smaller percentage for validation. However, if you have a small dataset, you may need to strike a balance between having enough data for training and enough for reliable model evaluation. Cross-validation is also a technique that can help you make the most of your data when you have limited amounts available for both training and validation.



# ANS TO QUESTION 2

The size of the train and validation sets can indeed affect how well you can predict the accuracy on the test set using the validation set. The key factors to consider are the representativeness of the validation set and the size of the training set:

Validation Set Representativeness:

Large Validation Set: A larger validation set is more likely to be representative of the overall data distribution. This means that the performance metric (e.g., accuracy) computed on the validation set is likely to provide a more accurate estimate of how well the model will perform on unseen data, including the test set. A large and representative validation set reduces the risk of obtaining overly optimistic or pessimistic estimates of model performance.

Small Validation Set: A small validation set may not capture the full diversity of the data distribution. It can be more susceptible to random fluctuations, making it less reliable for predicting performance on the test set. You might get more variability in validation performance estimates, which can make it harder to gauge how well your model generalizes.
2.Training Set Size:

Large Training Set: If you have a large training set, you can afford to allocate a larger portion of your data to the validation set without significantly reducing the amount of data available for training. This can help you obtain a more reliable estimate of model performance on the validation set while still having ample data for training. Predicting test set accuracy from a large validation set is generally more reliable.

Small Training Set: If your training set is small, you may need to allocate a smaller percentage to the validation set to ensure you have enough data for training. However, a smaller training set can make it challenging to train a model that generalizes well. In this case, predicting test set accuracy from the validation set becomes more challenging because the model may not be well-trained due to data scarcity.

In summary, a larger and representative validation set can provide a better estimate of how well your model will perform on the test set. However, it's crucial to strike a balance between validation set size and training set size, especially when dealing with limited data. Additionally, other factors such as the quality of the validation set (ensuring it is from the same distribution as the test set) and the overall dataset's diversity should be considered when predicting test set accuracy based on the validation set. Using techniques like cross-validation can also help in obtaining more reliable estimates of model performance when data is limited.

# ANS TO QUESTION 3

The choice of the percentage to reserve for the validation set should be based on several factors, including the size of your dataset, the nature of your problem, and your computational resources. There is no one-size-fits-all answer, but here are some general guidelines to help you strike a balance between having a representative validation set and a sufficiently large training set:

1.Common Split Ratios: In practice, a common split ratio is to allocate 70-80% of your data to the training set and 20-30% to the validation set. This is a reasonable starting point for many scenarios.

2.Small Datasets: If you have a very small dataset (e.g., a few hundred examples or less), you might need to allocate a larger portion to the training set. For example, you might consider an 80-20 or even a 90-10 split for training-validation.

3.Large Datasets: With a very large dataset (e.g., millions of examples), you can allocate a smaller percentage to the validation set (e.g., 10-20%) and still have a representative validation set.
4.Data Stratification: Ensure that your validation set is stratified, meaning it maintains the same class distribution as the overall dataset, especially if you have imbalanced classes. This helps ensure that the validation set accurately represents the data distribution.

5.Cross-Validation: If you have limited data and want to make the most of it, consider using cross-validation. Techniques like k-fold cross-validation allow you to partition your data into multiple train-validation splits and obtain more reliable performance estimates.

6.Iterative Experimentation: Sometimes, the best percentage for the validation set may need to be determined through iterative experimentation. You can try different split ratios and observe how your model performs on the validation set and, ultimately, on unseen test data.

7.Computational Resources: Consider your computational resources. If you have limited computational power, a larger validation set may result in longer training times. Conversely, if you have ample computational resources, you can allocate a larger portion to the validation set without significant overhead.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


# ANS TO QUESTION 1

Yes, averaging validation accuracy across multiple splits of your dataset can indeed give you more consistent and reliable results when evaluating the performance of a machine learning model. This technique is commonly referred to as "cross-validation."

Cross-validation helps in achieving more robust model evaluation by mitigating the potential bias introduced by a single train-test split. Here's how it works:
1.Data Splitting
2.Training and Testing
3.Averaging
 The key advantages of cross-validation are:
 1.Reduced Variance
 2.Better Generalization
 3.Use of All Data
 Common variants of cross-validation include k-fold cross-validation, stratified k-fold cross-validation (when dealing with imbalanced datasets), leave-one-out cross-validation (k equals the number of data points), and more. The choice of the specific cross-validation method depends on your dataset size, computational resources, and the nature of your problem.

In summary, averaging validation accuracy across multiple splits using cross-validation is a recommended practice to obtain more consistent and robust performance estimates for your machine learning models.

# ANS TO QUESTION 2

Cross-validation, while providing more robust and stable estimates of model performance, may not necessarily give a more accurate estimate of the test accuracy compared to a single train-test split. The primary purpose of cross-validation is to provide a better understanding of how well your model generalizes to unseen data and to reduce the impact of randomness or peculiarities in a single data split. Here's why:
1.Robustness
2.Better Data Utilization
However, it's important to note that the estimate of test accuracy obtained through cross-validation is still an estimate, and it may not precisely reflect the model's performance on completely unseen, real-world data. Here are some reasons why:
1.Data Leakage
2.Distributional Differences
3.Sample Size
While cross-validation is a valuable tool for model assessment and selection, it's essential to keep in mind that the ultimate goal is to deploy your model on new, unseen data. Therefore, it's often recommended to reserve a separate holdout test set, distinct from both the training and validation sets, to obtain a more accurate estimate of how well your model will perform in production.

# ANS TO QUESTION 3

The number of iterations or folds in cross-validation can have an impact on the quality of the estimate of your model's performance, but the relationship is not necessarily linear, and there are trade-offs to consider.

Here's how the number of iterations affects the estimate:
1.Fewer Iterations (Smaller k):

Advantages:
Computationally less expensive.
Faster to execute.
Each fold has a larger portion of the data, potentially leading to a more representative estimate of the overall performance if the dataset is large and diverse.
Disadvantages:
Higher variability: The estimate may be sensitive to the specific random splits, potentially leading to less stable results.
More Iterations (Larger k):

Advantages:
Reduced variability: With more folds, you obtain estimates based on multiple, smaller subsets of data, which can provide a more stable and reliable performance estimate.
.Disadvantages:
Computationally more expensive: As k increases, you need to train and evaluate the model multiple times, which can be computationally intensive.
Larger variance in individual fold performance estimates: With smaller subsets, each fold's estimate may be noisier or less representative of the overall performance.
The choice of the number of iterations (k) depends on your specific situation:

Standard Practice: 5-fold or 10-fold cross-validation is often considered a good compromise between computational cost and reliability. They are commonly used in practice.

Leave-One-Out Cross-Validation (LOOCV): In LOOCV, where k equals the number of data points, you get the least biased estimate but it can be computationally expensive and can lead to high variance in the estimates.

Stratified Cross-Validation: In cases of imbalanced datasets, stratified k-fold cross-validation can be beneficial to ensure that each fold maintains the class distribution of the original dataset.

In general, increasing the number of iterations (folds) can help reduce the variability in your cross-validation results and provide a more stable estimate of model performance. However, it comes at the cost of increased computation time. It's important to strike a balance between the number of iterations and computational resources available, keeping in mind that very high values of k may not always lead to significantly better estimates and may not be practical in all scenarios.

# ANS TO QUESTION 4

Increasing the number of iterations (folds) in cross-validation can help when dealing with a very small train dataset or validation dataset to some extent, but it has its limitations. Here's how it can affect the situation:

Dealing with a Small Train Dataset:
.Advantages:

With more iterations (higher k), you can ensure that each fold includes a larger portion of the data for training, which might be useful when you have a very small training dataset.
It can reduce the risk of overfitting on the small training data, as the model is trained and evaluated on multiple subsets of the data.
Disadvantages:

While increasing k can help mitigate overfitting to some extent, it doesn't magically create more data. If your training dataset is extremely small, cross-validation can only work with the limited data available, and there's a practical limit to how well the model can generalize.
Dealing with a Small Validation Dataset:

Increasing the number of iterations (higher k) won't directly address the issue of a small validation dataset. The size of the validation dataset in each fold remains the same, regardless of k.
In situations where both the train and validation datasets are very small, cross-validation may still provide some benefits in terms of model evaluation: